# DIET PROBLEM - PYOMO

*Zuria Bauer Hartwig* ([CAChemE](http://cacheme.org))

Sources:

* Original Problem: [Linear and Integer Programming](https://www.coursera.org/course/linearprogramming) (Coursera Course) - University of Colorado Boulder & University of Colorado System

* Based on the Examples from CAChemE's Optimization Workshop = [Taller-Optimizacion-Python-Pyomo](https://github.com/CAChemE/Taller-Optimizacion-Python-Pyomo) from [CAChemE.org](http://cacheme.org/optimizacion-programacion-matematica-con-python-pyomo/)

* [Neos Guide - The Diet Problem](http://www.neos-guide.org/content/diet-problem)

## Big picture

<img src="img/intro.png" alt="Dieta" style="width: 900px;"/>

The goal of the diet problem is to select a set of foods we can get at a famous fast food restaurant,
that will satisfy a set of daily nutritional requirement at minimum cost. 
The problem is formulated as a linear program where the objective is to minimize cost 
and the constraints are to satisfy the specified nutritional requirements. 
The diet problem constraints typically regulate the number of calories,
the calories from Fat, Total Fat, Cholesterol, Sodium, 
Carbohydrates, Fiber, Sugar, Proteine, Vitamine A and C,
Calcium and Sodium in the diet. 
While the mathematical formulation is simple, 
the solution may not be palatable! 
The nutritional requirements can be met without regard for taste or variety, 
so consider the output before digging into a meal from an "optimal" menu!

You can have a look in here:
https://www.mcdonalds.com/us/en-us/about-our-food/nutrition-calculator.html

**GDAs (Guideline Daily Amounts)** indicate the daily input of calories and nutrients (protein, fat, saturated fat , carbohydrates , sugars , salt and fiber) necessary for the daily diet of an adult or a child.


On the next figure you can see the recommended quantities for and adult, shown in solid color. It's also shown the quantities in one of the meals with which we are going to solve the diet problem.

<a href="http://www.foodlabel.org.uk/label/gda_values.aspx " target="_blank"><img src="img/GDA-table.png" alt="GDA-values" style="width: 800px;"/></a>

## Pyomo AML MODEL

#### 1. Import


In [1]:
from pyomo.environ import *
infinity = float('inf')

# We are importing Pyomo library to our Python enviroment

# Side-note:
# Usually importing all (*) the library is a VERY BAD idea
# By convention Pyomo documentation does that so here we are

#### 2. Data

We can use the following data:

- http://nutrition.mcdonalds.com/getnutrition/nutritionfacts.pdf (dead link)
- http://fastfoodnutrition.org/mcdonalds/chart
- http://www.fastfoodmenuprices.com/mcdonalds-prices/

In [2]:
# Let's have a look to the data:
# Mac or Linux ('!' meaning to be run through the terminal)
!cat data/foodmin1.dat

# Windows via cmd.exe
#!type data\foodmin.dat

# http://nutrition.mcdonalds.com/getnutrition/nutritionfacts.pdf
# http://www.fastfoodmenuprices.com/mcdonalds-prices/

#Food + Cost

param:  F:                                                  c  :=
"1 Low Fat Milk Jug"                                        1  
"Blueberry Pomegranate Smoothie"                            3.39  
"Coca-Cola Classic"                                         1.49  
"Dasani© Water"                                             1  
"Diet Coke"                                                 1.32  
"Diet Dr Pepper"                                            1.99  
"Dr Pepper"                                                 1.99  
"Fat Free Chocolate Milk Jug"                               2.29  
"Hi-C Orange Lavaburst"                                     1.59  
"Iced Tea"                                                  1  
"Mango Pineapple Smoothie"                                  2.39  
"McCafe Classic Lemonade"                                   1.79  
"Minu

**Note**: Window's users can use `!type data\foodmin.dat` command instead of `!cat data/foodmin.dat`

## Solving

#### 3. Model

In [3]:
# The AbstractModel class provides a context for defining and initializing abstract optimization models in Pyomo

model = AbstractModel()

#### 4. Sets

In [4]:
## DEFINE SETS
# Set data that is used to define a model instance

# Products
model.F = Set()
# Nutrients
model.N = Set()

#### 5. Parameters

In [5]:
## DEFINE PARAMETERS
# Parameter data that is used to define a model instance

# Cost
model.c = Param(model.F, within = PositiveReals)

# Amount of nutrient
model.a    = Param(model.F, model.N, within = NonNegativeReals)

# Max and Min for each Nutrient
model.Nmin = Param(model.N, within = NonNegativeReals, default = 0.0)
model.Nmax = Param(model.N, within = NonNegativeReals, default = infinity)

#### 6. Variables

In [6]:
## VARIABLES
# Decision variables in a model

# Number of servings (notice we are using integers)
model.x = Var(model.F, within = NonNegativeIntegers)

#### 7. Objective

In [7]:
## OBJECTIVE
# Expressions that are minimized or maximized in a model

# Minimize z(cost)
def cost(model):
    return sum(model.c[i] * model.x[i] for i in model.F)
model.cost = Objective(rule=cost)

#### 8. Constraints

In [8]:
## CONSTRAINTS
# Constraint expressions that impose restrictions on variable values in a model

#Max
def nutrients_max(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return value <= model.Nmax[j]
model.nutrient_limit_max = Constraint(model.N, rule=nutrients_max)

#Min
def nutrient_min(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return model.Nmin[j] <= value 
model.nutrient_limit_min = Constraint(model.N, rule=nutrient_min)

## Solving a model

In [9]:
#To get a solution:

# Mac or Linux ('!' meaning to be run through the terminal)
!pyomo solve --solver=glpk problems/diet-minimize/minimizecost.py data/foodmin1.dat

# windows:
#!pyomo solve --solver=glpk problems\diet-minimize\minimizecost.py data\foodmin.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.15] Applying solver
[    0.18] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 4.39
    Solver results file: results.yml
[    0.18] Applying Pyomo postprocessing actions
[    0.18] Pyomo Finished


##  Results

Lets have a look to the results now since...
**We are HUNGRY!**

In [10]:
# Mac or Linux ('!' meaning to be run through the terminal)
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4.39
  Upper bound: 4.39
  Number of objectives: 1
  Number of constraints: 26
  Number of variables: 116
  Number of nonzeros: 2339
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.0071887969970703125
# ----------------------------------------------------------
#   Solution Information
# -------------------------------

The solution we have obtained using Pyomo is a **Artisan Grilled Chicken Sandwich** which has the folliwing input of calories and nutrients for $4.39

<img src="img/artisan.jpg" style="width: 400px;"/>
<img src="img/cheddar1.png" alt="Burger" style="width: 800px;"/>